# MNIST Neural Network

Working from this [kernel in Kaggle](https://www.kaggle.com/sdelecourt/cnn-with-pytorch-for-mnist)

In [81]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [82]:
# Trying to keep training parameters in the same place
BATCH_SIZE = 32
EPOCHS = 5

In [83]:
df = pd.read_csv('digit-recognizer/train.csv')
print(df.shape)

(42000, 785)


In [84]:
y = df['label'].values
X = df.drop(['label'],1).values # Drop the labels so you don't get data pollution

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15) # Split into test and training data

In [85]:
torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long; But why?

# create feature and targets tensor for test set
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

It looks like the data flow in torch in this example goes follows:
1. Read the data in from csv file
2. Convert the data to numpy array (can be combined with step 1)
3. Split the dataset into test and train data, using scikit learn functionality
4. Convert the test and train datasets into *Torch Tensor*
5. Convert the *Torch Tensors* into *Tensor Datasets* for both train and test data
6. Input the Datasets into a data loader, considering a specified batch size

Seems like an unnecessary number of steps; Chance to optimize, at the very least for shorter and clearer code.

In [87]:
# Same as other example; Create an inherited class for the neural network
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,10) # Linear means fully-connected layer
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 
mlp = MLP()
print(mlp)

MLP(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)


'*' Copied from the example code

Keep in mind that none of this network involves convolution; All layers are fully-connected

We have 784*(250+1) + 250*(100+1) + 100*(10+1) = 222 360 parameters to train

In [88]:
def fit(model, train_loader):
    # Adam is a method for stochastic gradient descent; avail. below
    # https://pytorch.org/docs/stable/_modules/torch/optim/adam.html
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss() # Cross Entropy loss is a loss fxn, AKA log loss
    # Read more about cross entropy, or log loss, here: https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html
    model.train() # Calling .train() method on nn.Module object
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            # print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
                

Run the model

In [91]:
fit(mlp, train_loader)

Epoch : 0 [0/35700 (0%)]	Loss: 0.091644	 Accuracy:96.875%
Epoch : 0 [1600/35700 (4%)]	Loss: 0.536852	 Accuracy:96.507%
Epoch : 0 [3200/35700 (9%)]	Loss: 0.028471	 Accuracy:96.349%
Epoch : 0 [4800/35700 (13%)]	Loss: 0.083459	 Accuracy:96.482%
Epoch : 0 [6400/35700 (18%)]	Loss: 0.305395	 Accuracy:96.315%
Epoch : 0 [8000/35700 (22%)]	Loss: 0.450834	 Accuracy:96.352%
Epoch : 0 [9600/35700 (27%)]	Loss: 0.045381	 Accuracy:96.408%
Epoch : 0 [11200/35700 (31%)]	Loss: 0.003097	 Accuracy:96.359%
Epoch : 0 [12800/35700 (36%)]	Loss: 0.199553	 Accuracy:96.462%
Epoch : 0 [14400/35700 (40%)]	Loss: 0.106324	 Accuracy:96.529%
Epoch : 0 [16000/35700 (45%)]	Loss: 0.255821	 Accuracy:96.544%
Epoch : 0 [17600/35700 (49%)]	Loss: 0.060047	 Accuracy:96.586%
Epoch : 0 [19200/35700 (54%)]	Loss: 0.047019	 Accuracy:96.553%
Epoch : 0 [20800/35700 (58%)]	Loss: 0.047048	 Accuracy:96.597%
Epoch : 0 [22400/35700 (63%)]	Loss: 0.005376	 Accuracy:96.594%
Epoch : 0 [24000/35700 (67%)]	Loss: 0.119014	 Accuracy:96.605%
Epoch

In [101]:
def evaluate(model):
#model = mlp
    correct = 0 
    for test_imgs, test_labels in test_loader:
#         print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
#         print(test_imgs.shape)
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))
evaluate(mlp)

Test accuracy:0.957% 


In [93]:
#### Sam Exploration
# I want to run this model on an image of a digit and see, in near realtime, what it predicts
from PIL import Image
import os
from matplotlib import pyplot as plt
from IPython.display import display
from ipywidgets import interact # Interactive IPython is badass; Highly recommend. More info on setup here: https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html

SAMPLE_JPEGS = 'digit-recognizer/testSample/'
SAMPLE_JPEGS = [SAMPLE_JPEGS +'/' +i for i in os.listdir(SAMPLE_JPEGS)]
# im = Image.open(SAMPLE_JPEGS[0])
# display(im)
@interact
def test_model(im_index = 100):
    """
    Tests the Computer vision model in namespace as 'mlp', using an image read from the im_file filepath
    """
    im_file = SAMPLE_JPEGS[im_index]
    # Gets and displays the image 
    im = Image.open(im_file)
    im_arr = np.array(im)
    plt.imshow(im_arr, cmap ='binary')
#     plt.show()
    
    # Ravel the array to get something to feed to the classifier neural net
    im_arr = im_arr.ravel()
    im_arr_test = torch.Tensor(im_arr).float()
    im_arr_test = im_arr_test.unsqueeze(0) # Adds a dimension to the structure with size 1 at index 0; semantics
#     print(im_arr_test.shape)
    out = mlp(im_arr_test)
    pred = torch.max(out,1)[1][0]
    print('\n\n\nThe model predicted {} for the below image\n'.format(pred))
    

interactive(children=(IntSlider(value=100, description='im_index', max=300, min=-100), Output()), _dom_classes…

# Adding Convolution to the MLP 

In [98]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Define all of the layers in the __init__ method
        # Conv2d is a class, so you are instantiating classes here? 
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        # Define how the data will flow through the CNN
        x = F.relu(self.conv1(x)) # Interesting how .conv1, defined in init, is a function
        #x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training) # I expect dropouts ~ pooling layers
        # View? Are we looking at the
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
cnn = CNN()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
# print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [99]:
fit(cnn,train_loader)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 32 1, but got 2-dimensional input of size [32, 784] instead

In [39]:
# help(mlp.eval())
mlp.eval()

MLP(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)